In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from tqdm.notebook import tqdm
from typing import Optional
from quapopt import ancillary_functions as anf
from quapopt.data_analysis.data_handling import (CoefficientsType,
                                                 CoefficientsDistribution,
                                                 CoefficientsDistributionSpecifier,
                                                 HamiltonianModels)
from quapopt.hamiltonians.generators import build_hamiltonian_generator, ERDOS_RENYI_TYPES

### Generate random Hamiltonians and save them to files

* We will generate random Hamiltonians from a given model and save them to files.
* We can generate various classes, specified by HamiltonianModels enum.
* Instances can be generated on the fly or read from file.
* We can also solve the Hamiltonian classically (default solver is Burer-Monteiro rank-2 relaxation), to get the ground state energy and the highest energy. Those results can be saved to file.



In [ ]:
# Choose hamiltonia model
hamiltonian_model = HamiltonianModels.SherringtonKirkpatrick
#choose system sizes to generate
full_noq_range = [16]
#choose seeds for instances generation
full_seeds_range = list(range(0, 3))

if hamiltonian_model in [HamiltonianModels.LABS]:
    print("FOR LABS, we have only single instance per system size. Changing seeds range to [0]")
    full_seeds_range = [0]

coefficients_type = CoefficientsType.DISCRETE
coefficients_distribution = CoefficientsDistribution.Uniform
coefficients_distribution_properties = {'low': -1, 'high': 1, 'step': 1}

coefficients_distribution_specifier = CoefficientsDistributionSpecifier(CoefficientsType=coefficients_type,
                                                                        CoefficientsDistributionName=coefficients_distribution,
                                                                        CoefficientsDistributionProperties=coefficients_distribution_properties)

arguments_dict_builder = {}

#We need to specify kwargs for builder = hamiltonian generator constructor. Accepts arguments:
#    hamiltonian_model: HamiltonianModels -- which model to implement?
#    coefficients_distribution_specifier -- specifying how to sample interaction coefficients
#    localities -- tuple of int, specifying which localities to include (1 for local fields, 2 for 2-body interactions, etc)
#    erdos_renyi_type: ERDOS_RENYI_TYPES -- specific for erdor-renyi graph, whether using "GnP" or "GnM" model
HAMILTONIAN_BUILDER_CONFIGS = { #choose whether to include local fields for SK model
                                 HamiltonianModels.SherringtonKirkpatrick: {'localities':(2,)},
                                  #regular graphs typically don't include local fields
                                 HamiltonianModels.RegularGraph: {'localities': (2,)},
                                #choose what localities to include for ErdosRenyi graph
                                 HamiltonianModels.ErdosRenyi: {'localities': (1,2),
                                                                  'erdos_renyi_type': ERDOS_RENYI_TYPES.Gnp} }

def _get_additional_instance_arguments(hamiltonian_model:HamiltonianModels,
                                       number_of_qubits:Optional[int]=None,
                                       ):
    """
    This function is used to get additional arguments for instance generation. It allows to make some parameters of the instance to depend on the system size.
    For example -- one might want to generate a graph with a given average degree for MaxCut model.
    """

    arguments_instance = {}
    if hamiltonian_model in [HamiltonianModels.RegularGraph]:
        graph_degree = 3
        arguments_instance['graph_degree'] = graph_degree
    elif hamiltonian_model in [HamiltonianModels.ErdosRenyi]:
        edge_probability = 0.01
        arguments_instance['p_or_M'] = edge_probability
    elif hamiltonian_model in [HamiltonianModels.MaxCut]:
        average_degree = number_of_qubits // 2
        edge_probability = average_degree / (number_of_qubits - 1)
        arguments_instance['p_or_M'] = edge_probability
    elif hamiltonian_model in [HamiltonianModels.MAX2SAT]:
        clause_density = 1.1
        arguments_instance['clause_density'] = clause_density

    return arguments_instance


#create hamiltonian generator
generator_cost_hamiltonian = build_hamiltonian_generator(hamiltonian_model=hamiltonian_model,
                                                         coefficients_distribution_specifier=coefficients_distribution_specifier,
                                                         **HAMILTONIAN_BUILDER_CONFIGS[hamiltonian_model])



In [ ]:
#whether to find ground state energy and highest energy by solving the hamiltonian classically
solve_hamiltonians = True
solver_kwargs = {'solver_name': "BURER2002",
                 'solver_timeout': 1}
#whether to save hamiltonians to files
save_hamiltonians = True


for number_of_qubits in tqdm(full_noq_range):
    anf.cool_print("NUMBER OF QUBITS", number_of_qubits)

    for seed_hamiltonian in full_seeds_range:
        print("INSTANCE:", seed_hamiltonian)

        #potentially, we get additional pre-configured arguments for instance generation
        arguments_instance = _get_additional_instance_arguments(hamiltonian_model=hamiltonian_model,
                                                                number_of_qubits=number_of_qubits)
        cost_hamiltonian = generator_cost_hamiltonian.generate_instance(number_of_qubits=number_of_qubits,
                                                                        seed=seed_hamiltonian,
                                                                        read_from_drive_if_present=True,
                                                                        **arguments_instance
                                                                        )
        if solve_hamiltonians:
            #This will write known solutions if they already exist in instance attributes (if not, it doesn't do anything)
            cost_hamiltonian.write_solutions_to_file()
            #This solving is required to get lowest energy state and highest energy state
            cost_hamiltonian.solve_hamiltonian(both_directions=True, #both directions means whether to also find highest energy state
                                               solver_kwargs=solver_kwargs)

        ground_state_energy = cost_hamiltonian.ground_state_energy
        highest_energy = cost_hamiltonian.highest_energy

        anf.cool_print("GROUND STATE ENERGY", ground_state_energy)
        anf.cool_print("HIGHEST ENERGY", highest_energy)
        if save_hamiltonians:
            cost_hamiltonian.write_to_file()



In [ ]:
print("Okay, then")